In [ ]:
from brtdevkit.data import Dataset
prod_dataset = Dataset.retrieve(name='mannequin_in_dust_v2')
new_dataset = Dataset.retrieve(id='62913b27f284f6f9783716a2')
prod_df = prod_dataset.to_dataframe()
new_df = new_dataset.to_dataframe()
id1 = list(prod_df['id'])
id2 = list(new_df['id'])
print(len(id1), len(id2))
all_ids = id1 + id2

In [ ]:
print(prod_dataset.description)

In [ ]:
sum(new_df['operation_time'] == 'unknown')

In [ ]:
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
imageids_to_dataset(all_ids, 'mannequin_in_dust_v2_plus', dataset_description="A mannequin in several positions with dust billowing around. ~5k images", dataset_kind=Dataset.KIND_ANNOTATION, production_dataset=False)

# Get 5th vehicle sequece

In [ ]:
from brtdevkit.data import Dataset
ds_aletheia = Dataset.retrieve(name='vehicles_driving_through_dust_1_2_5_hard')
df = ds_aletheia.to_dataframe()

In [ ]:
import pandas as pd
df.collected_on_dt = pd.to_datetime(df.collected_on)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime
dt = datetime.datetime(2023,5,1)
df['collected_on_dt'] = pd.to_datetime(df.collected_on)
df_5 = df[df.collected_on_dt > dt]

In [ ]:
import re
import numpy as np
from pathlib import Path
def guess_missing_operation_time_from_special_notes(row: pd.Series) -> str:
    """Some data (suv_driving_through_rear_dust_anno) didn't have operation time
    in the rows (or latitude/longitude), so you can't easily infer the
    time of day. However, it can be guessed from the special notes
    for these particular bags, so its ok.

    Args:
        row: should have 'special_notes' and 'operation_time' columns

    Returns:
        str: the operation time
    """
    notes = str(row['special_notes']).lower()
    notes = row['special_notes'].lower() if isinstance(row['special_notes'], str) else "unknown"
    if re.search(r'dust_pos\d(-atmp-2)?', notes):
        return 'dawn_dusk'
    elif re.search(r'dust-dusk-pos[1-5]', notes):
        return 'dawn_dusk'
    elif re.search(r'\bnight\b', notes):
        return 'nightime' # that's how it's spelled in the database
    elif re.search(r'\bdusk\b', notes):
        return 'dawn_dusk'
    elif re.search(r'\bday\b', notes):
        return 'daytime'
    else:
        return row['operation_time']

def fill_missing_operation_time(df: pd.DataFrame) -> pd.Series:
    """Some data  didn't have operation time in the rows (or latitude/longitude),
    so you can't read the time of day. It is recorded in another file that we must read from."""
    fpath =  "/home/alexli/git/JupiterScripts/apps/embedded_metrics/data/operation_time_for_sequence_data.npz"
    sequences = np.load(fpath, allow_pickle=True)
    for operation_time, sequence in sequences.items():
        seq_ids = np.concatenate(sequence)
        df.loc[df.id.isin(list(seq_ids)), 'operation_time'] = operation_time
    df['operation_time'] = df.apply(guess_missing_operation_time_from_special_notes, axis=1)
    return df['operation_time']
op_time_computed = fill_missing_operation_time(df_5)

In [ ]:
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
imageids_to_dataset(image_ids=list(df_5['id']), 'vehicles_driving_through_dust_5', dataset_description="Vehicles driving in dust, 2274 images", dataset_kind=Dataset.KIND_ANNOTATION, production_dataset=False)